In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import pandas as pd
import numpy as np
import random as rd
import matplotlib.pyplot as plt

In [5]:
dataset =pd.read_csv("../input/customer-personality-analysis/marketing_campaign.csv", sep="\t")
dataset.head()

In [6]:
dataset.isnull().sum()

In [7]:
dataset=dataset.dropna()

In [10]:

X = dataset[["Income","Education"]]
#Visualise data points
plt.scatter(X["Income"],X["Education"],c='black')
plt.xlabel('Income')
plt.ylabel('Education ')
plt.show()

In [14]:
K=3

# Select random observation as centroids
Centroids = (X.sample(n=K))
plt.scatter(X["Income"],X["Education"],c='black')
plt.scatter(Centroids["Income"],Centroids["Education"],c='red')
plt.xlabel('Income')
plt.ylabel('Education')
plt.show()

In [15]:
K=3
color=['blue','green','cyan']
for k in range(K):
    data=X[X["Cluster"]==k+1]
    plt.scatter(data["Income"],data["Education"],c=color[k])
plt.scatter(Centroids["Income"],Centroids["Education"],c='red')
plt.xlabel('Income')
plt.ylabel('Education')
plt.show()

In [16]:
# calculating the no of days as customers
dataset['c_date'] = '01-01-2015'
dataset['Dt_Customer'] =pd.to_datetime(dataset.Dt_Customer)
dataset['c_date'] = pd.to_datetime(dataset.c_date)
dataset['days_customer'] = (dataset['c_date']-dataset['Dt_Customer']).dt.days

# defining accepted_any_cmp if customer have ever taken any campaign offer ( 1- yes, 0-no)
dataset['accepted_any_cmp']= dataset['AcceptedCmp1']+dataset['AcceptedCmp2']+dataset['AcceptedCmp3']+dataset['AcceptedCmp4']+dataset['AcceptedCmp5']+dataset['Response']
dataset['accepted_any_cmp']= np.where(dataset['accepted_any_cmp'] > 0, 1, 0)

# defining martial status as ( 0- single adult, 1- two adults)
mapping = {'Single' : 0, 'Together': 1, 'Married': 1, 'Divorced': 0, 'Widow': 0, 'Alone': 0,
       'Absurd': 0, 'YOLO': 0, 'Graduation': 1, 'PhD': 2, 'Master': 2, 'Basic': 0, '2n Cycle': 2}
dataset=dataset.replace({'Marital_Status': mapping, 'Education': mapping}) 

dataset['kid_teen'] = dataset['Kidhome'] + dataset['Teenhome']
dataset['kid_teen']= np.where(dataset['kid_teen'] > 0, 1, 0)

dataset.columns

In [17]:
dataset=dataset[['Education', 'Marital_Status', 'Income', 'kid_teen',
       'Recency','NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth', 'accepted_any_cmp', 
       'Complain','days_customer']]

In [18]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(dataset)

In [19]:
from sklearn.cluster import KMeans
wcss = []
for i in range(1,16):
    kmeans = KMeans(n_clusters = i, init = 'k-means++', random_state = 42)
    kmeans.fit(X)
    wcss.append(kmeans.inertia_)
    
plt.plot(range(1,16), wcss)
plt.title('method')
plt.xlabel('Number of clusters')
plt.ylabel('wcss')
plt.show()

In [21]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 2)
X = pca.fit_transform(X)

In [22]:
kmeans = KMeans(n_clusters = 5, init = 'k-means++', random_state = 42)
y_kmeans = kmeans.fit_predict(X)

In [23]:
print(y_kmeans)

In [25]:
plt.scatter(X[y_kmeans ==0, 0], X[y_kmeans == 0, 1], s = 100, c = 'red', label='A')
plt.scatter(X[y_kmeans ==1, 0], X[y_kmeans == 1, 1], s = 100, c = 'blue', label='B')
plt.scatter(X[y_kmeans ==2, 0], X[y_kmeans == 2, 1], s = 100, c = 'green', label='C')
plt.scatter(X[y_kmeans ==3, 0], X[y_kmeans == 3, 1], s = 100, c = 'cyan', label='D')
plt.scatter(X[y_kmeans ==4, 0], X[y_kmeans == 4, 1], s = 100, c = 'gray', label='E')
plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:,1], s = 300, c = 'yellow', label = 'centroids')
plt.title('The clusters of Customers')
plt.xlabel('Component1')
plt.ylabel('Component2')
plt.legend()
plt.show()

Comment:
We can use the data to figure out the information of customers visiting store and how much are they educated;
As profession or level of study always aims customers choice of shops
Secondly We can figue out the income of customers and compare that to their daily visits
Thirdly we can figure out whether the Kids living in the house increases the store and purchase visit or not!